<a href="https://colab.research.google.com/github/AndresMontesDeOca/Zubale/blob/main/Mid_LevelDataEngineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Libreries

In [13]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Challenge 1: CSV File Manipulation

## 1.2 Load Files

In [14]:
# Products
data_products = pd.read_csv('products.csv')

# Rename Index
data_products.rename(columns={'id': 'product_id'}, inplace=True)

print(data_products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  20 non-null     int64  
 1   name        20 non-null     object 
 2   category    20 non-null     object 
 3   price       20 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 772.0+ bytes
None


In [15]:
# Orders
data_orders = pd.read_csv('orders.csv', parse_dates=['created_date'])

# Rename Index
data_orders.rename(columns={'id': 'order_id'}, inplace=True)

print(data_orders.info(), '\b')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      50 non-null     int64         
 1   product_id    50 non-null     int64         
 2   quantity      50 non-null     int64         
 3   created_date  50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.7 KB
None 


## 1.3 EDA Products

In [31]:
# Dataframe overview
display(data_products.head())
display(data_products.tail())

# Product ID relates with Product Name

,product_id,name,category,price
0,1,Product_1,Pants,92.55
1,2,Product_2,Shirts,43.11
2,3,Product_3,Jackets,59.02
3,4,Product_4,Shoes,49.65
4,5,Product_5,Pants,44.59


,product_id,name,category,price
15,16,Product_16,Dresses,38.08
16,17,Product_17,Shirts,98.51
17,18,Product_18,Jackets,10.11
18,19,Product_19,Shirts,51.94
19,20,Product_20,Jackets,37.85


In [32]:
# Not repeated products
print('Product id unique?:', data_products.index.is_unique)
print('Product name unique?:', data_products['name'].is_unique, '\n')

# Categories
print(data_products['category'].value_counts())

Product id unique?: True
Product name unique?: True 

category
Shirts     6
Pants      4
Jackets    4
Shoes      3
Dresses    3
Name: count, dtype: int64


## 1.4 EDA Orders

In [18]:
# Dataframe overview
display(data_orders.head())

,order_id,product_id,quantity,created_date
0,1,11,1,2024-12-01
1,2,17,2,2024-12-01
2,3,19,1,2024-12-01
3,4,12,1,2024-12-01
4,5,11,2,2024-12-01


In [19]:
# Not repeated products
print('Order id unique?:', data_orders.index.is_unique)

# Each order relates with a single Product ID

Order id unique?: True


## 1.5 Merge

In [20]:
# Merge the DataFrames using product_id and index
data_merged = pd.merge(data_orders, data_products, left_on='product_id', right_on='product_id')

# Total Price Calculation
data_merged['total_price'] = data_merged['price'] * data_merged['quantity']

# Columns Rename
data_merged.rename(columns={'created_date': 'order_created_date', 'name':'product_name'}, inplace=True)

# Final Dataframe
data_final = data_merged[['order_created_date', 'order_id', 'product_name', 'quantity', 'total_price']]

display(data_final.head())

,order_created_date,order_id,product_name,quantity,total_price
0,2024-12-01,1,Product_11,1,69.06
1,2024-12-01,2,Product_17,2,197.02
2,2024-12-01,3,Product_19,1,51.94
3,2024-12-01,4,Product_12,1,50.99
4,2024-12-01,5,Product_11,2,138.12


## 1.6 Export to CSV

In [21]:
data_final.to_csv('order_full_information.csv', index=False)

# Challenge 2

## 2.0 Libraries

In [22]:
!pip install freecurrencyapi

import freecurrencyapi


client = freecurrencyapi.Client('fca_live_PXKARG5rRSKzbUPWjK6nGHl61Q70AoCzuqWTkwSo')
result = client.currencies(currencies=['USD', 'BRL'])

print(result)

  Preparing metadata (setup.py) ... done
  Created wheel for freecurrencyapi: filename=freecurrencyapi-0.1.0-py3-none-any.whl size=2610 sha256=cbd5136fdb265bd8cdb649214af9206b13fb89127fefd47823758116f37cc1c2
  Stored in directory: /root/.cache/pip/wheels/c3/40/bc/cb8b71d6a508b946b573c20a379370e6cca0229e094d86e5ad
Successfully built freecurrencyapi
{'data': {'USD': {'symbol': '$', 'name': 'US Dollar', 'symbol_native': '$', 'decimal_digits': 2, 'rounding': 0, 'code': 'USD', 'name_plural': 'US dollars', 'type': 'fiat'}, 'BRL': {'symbol': 'R$', 'name': 'Brazilian Real', 'symbol_native': 'R$', 'decimal_digits': 2, 'rounding': 0, 'code': 'BRL', 'name_plural': 'Brazilian reals', 'type': 'fiat'}}}


## 2.1: Currency Conversion

In [23]:

rates = client.latest()

# Filter the dictionary for USD and BRL
filtered_result = {k: v for k, v in rates['data'].items() if k in ['USD', 'BRL']}
rate_brl = filtered_result['BRL']
rate_usd = filtered_result['USD']

print(filtered_result)

{'BRL': 5.9235108979, 'USD': 1}


### 2.1.2: Prepare the new Dataframe

In [24]:
# Check if the rate is not 0
if rate_brl != 0:
  data_final['total_price_us'] = data_final['total_price'] / rate_brl

data_final.rename(columns={'total_price': 'total_price_br'}, inplace=True)

display(data_final.head())

<ipython-input-24-4fe870ba9d14>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['total_price_us'] = data_final['total_price'] / rate_brl
<ipython-input-24-4fe870ba9d14>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final.rename(columns={'total_price': 'total_price_br'}, inplace=True)


,order_created_date,order_id,product_name,quantity,total_price_br,total_price_us
0,2024-12-01,1,Product_11,1,69.06,11.658626
1,2024-12-01,2,Product_17,2,197.02,33.260680
2,2024-12-01,3,Product_19,1,51.94,8.768448
3,2024-12-01,4,Product_12,1,50.99,8.608071
4,2024-12-01,5,Product_11,2,138.12,23.317253


### 2.1.3: Export to CSV

In [25]:
data_final.to_csv('fixed_order_full_information.csv', index=False)

## 2.2: Data Exploration with Python


In [26]:
#Upload the results in a file named kpi_product_orders.csv

### 2.2.1: The date with the highest number of orders

In [27]:
# Top day of sales
orders_by_date = data_final.groupby('order_created_date')['order_id'].count().reset_index(name='orders_count')
orders_by_date.sort_values(by='orders_count', ascending=False, inplace=True)
display(orders_by_date.head(1))

# There are no sales on Dec 4th and Dec 8th
orders_by_date.sort_values(by='order_created_date')

,order_created_date,orders_count
4,2024-12-06,10


,order_created_date,orders_count
0,2024-12-01,8
1,2024-12-02,5
2,2024-12-03,3
3,2024-12-05,6
4,2024-12-06,10
5,2024-12-07,4
6,2024-12-09,7
7,2024-12-10,7


### 2.2.2: The most sold product and its total sales value

In [28]:
product_sales = data_final.groupby('product_name')['total_price_us'].sum()

most_sold_product = product_sales.idxmax()  # Get the product name with the maximum sales value
total_sales_value = np.round(product_sales.max(), 2)  # Get the corresponding total sales value


print(f'The most sold product is {most_sold_product} with a total sales value of U$S{total_sales_value}')


The most sold product is Product_7 with a total sales value of U$S185.63


### 2.2.3: The top 3 most demanded categories

In [29]:
# Fusionar los datos de orders y products
merged_data = data_final.merge(data_products, left_on='product_name', right_on='name')

# Calcular las cantidades totales por categoría
category_demand = merged_data.groupby('category')['quantity'].sum()

# Obtener las 3 categorías más demandadas
top_categories = category_demand.sort_values(ascending=False).head(3)

print("Top 3 most demanded categories:")
print(top_categories)


Top 3 most demanded categories:
category
Shirts     50
Jackets    30
Pants      29
Name: quantity, dtype: int64
